# School Data Analysis

A high-level snapshot of the district's key metrics, presented in a table format
An overview of the key metrics for each school, presented in a table format
Tables presenting each of the following metrics:
Top 5 and bottom 5 performing schools, based on the overall passing rate
The average math score received by students in each grade level at each school
The average reading score received by students in each grade level at each school
School performance based on the budget per student
School performance based on the school size 
School performance based on the type of school

In [5]:
# Add the Pandas dependency.
import pandas as pd, os


# Files to load
school_data_to_load = os.path.join("Resources","schools_complete.csv")
student_data_to_load = os.path.join("Resources","students_complete.csv")

# Read the school data file and store it in a Panda Dataframe
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Put the student names in a list
student_names = student_data_df["student_name"].tolist()

# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the "prefixes_suffixes" list and replace them with an empty space, "" when it occurs
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")



In [48]:
# Combine the data into a single dataset.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

# Get the total number of students
student_count = school_data_complete_df["Student ID"].count()

# Calculate the total number of schools.
school_count = school_data_df["school_name"].count()

# Calculate the total budget
total_budget = school_data_df["budget"].sum()



# Calculate average reading and average math scores
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

# Calcuate the number of students passing math.
passing_reading_count = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]["Student ID"].count()

# Calcuate the number of students passing reading.
passing_math_count = school_data_complete_df[school_data_complete_df["math_score"] >= 70]["Student ID"].count()

# Calculate the percentage of students who passed math
passing_math_percentage = passing_math_count/float(student_count) * 100

# Calculate the percentage of students who passed reading
passing_reading_percentage = passing_reading_count/float(student_count) * 100

# Calculate the students who passes both math and reading.
overall_passing_math_reading_count = school_data_complete_df[(school_data_complete_df["math_score"]>=70) & (school_data_complete_df["reading_score"]>=70)]["Student ID"].count()

# Calcuate the overall passing percentage.
overall_passing_percentage = overall_passing_math_reading_count/float(student_count)*100

# Add a list of values with keys to create a new DataFrame.
district_summary_df = pd.DataFrame(
    [{"Total Schools": school_count,
          "Total Students": student_count,
          "Total Budget": total_budget,
          "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])


# Format the data in the District Summary DataFrame
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)



,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90


In [68]:
# Determine the school type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Determine the school student count
per_school_counts = student_data_df["school_name"].value_counts()


# Calculate the school budget
per_school_budget = school_data_df.set_index(["school_name"])["budget"]

# Calcuate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the math scores.
per_school_math_average = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]

# Calculate the reading scores.
per_school_reading_average = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]

per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both math and reading
per_overall_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)&(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students who passed both math and reading.
per_overall_passing_math_reading = per_overall_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the overall passing percentage.
per_overall_passing_percentage = per_school_passing_math_reading / per_school_counts * 100




# adding a list of values with ketys to create a new DataFrame
per_school_summary_df = pd.DataFrame({
     "School Type": per_school_types,
     "Total Students": per_school_counts,
     "Total School Budget": per_school_budget,
     "Per Student Budget": per_school_capita,
     "Average Math Score": per_school_math_average,
     "Average Reading Score": per_school_reading_average,
     "% Passing Math": per_school_passing_math,
     "% Passing Reading": per_school_passing_reading,
     "% Overall Passing": per_overall_passing_percentage})

# Format the data in the District Summary DataFrame
per_school_summary_df["Total Students"] = per_school_summary_df["Total Students"].map("{:,}".format)
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:.1f}".format)
per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:.1f}".format)
per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:.0f}".format)
per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:.0f}".format)
per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:.0f}".format)

per_school_summary_df.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,67,82,1
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94,97,5
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66,81,2
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68,79,2
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93,97,6
